In [1]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [2]:
# read in with pandas
winter1 = pd.read_csv ('iag20-21data.txt')
winter2 = pd.read_csv ('iag21-22data.txt')
winter3 = pd.read_csv ('iag22-23data.txt')

In [3]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [4]:
# mask out days without precipitation
codes = ['GS','IC','PL','SG','SN']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [5]:
precip_days2

,station,datetime,precip(mm),wxcodes
137,IAG,2021-12-05 18:39:00,T,-SN
138,IAG,2021-12-05 18:53:00,T,-SN
177,IAG,2021-12-06 22:16:00,T,-SN
184,IAG,2021-12-07 04:53:00,T,-SN
185,IAG,2021-12-07 05:53:00,T,-SN
...,...,...,...,...
3259,IAG,2022-02-27 19:27:00,T,-SN
3260,IAG,2022-02-27 19:33:00,T,-SN
3261,IAG,2022-02-27 19:43:00,T,-SN
3262,IAG,2022-02-27 19:49:00,T,-SN


In [6]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_35136\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_35136\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_35136\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [7]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_35136\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_35136\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_35136\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [8]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 53]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 53]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 53]

In [9]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [10]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [11]:
iag = LocationInfo(
    'Niagra Falls' 'New York', 
    latitude= 43.0962,
    longitude= -79.0377, 
    timezone='America/New_York'
)

iag

LocationInfo(name='Niagra FallsNew York', region='England', timezone='America/New_York', latitude=43.0962, longitude=-79.0377)

In [12]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(iag.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [13]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(iag.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [14]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(iag.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [15]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 07:28:48  2020-12-01 16:41:48  2020-12-01
1   2020-12-02 07:29:52  2020-12-02 16:41:32  2020-12-02
2   2020-12-03 07:30:55  2020-12-03 16:41:18  2020-12-03
3   2020-12-04 07:31:56  2020-12-04 16:41:06  2020-12-04
4   2020-12-05 07:32:56  2020-12-05 16:40:57  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 07:00:02  2021-02-24 17:59:09  2021-02-24
86  2021-02-25 06:58:27  2021-02-25 18:00:26  2021-02-25
87  2021-02-26 06:56:50  2021-02-26 18:01:42  2021-02-26
88  2021-02-27 06:55:13  2021-02-27 18:02:58  2021-02-27
89  2021-02-28 06:53:35  2021-02-28 18:04:14  2021-02-28

[90 rows x 3 columns]


In [16]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 07:28:32  2021-12-01 16:41:52  2021-12-01
1   2021-12-02 07:29:36  2021-12-02 16:41:35  2021-12-02
2   2021-12-03 07:30:40  2021-12-03 16:41:21  2021-12-03
3   2021-12-04 07:31:41  2021-12-04 16:41:08  2021-12-04
4   2021-12-05 07:32:42  2021-12-05 16:40:59  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 07:00:25  2022-02-24 17:58:51  2022-02-24
86  2022-02-25 06:58:50  2022-02-25 18:00:07  2022-02-25
87  2022-02-26 06:57:14  2022-02-26 18:01:24  2022-02-26
88  2022-02-27 06:55:37  2022-02-27 18:02:40  2022-02-27
89  2022-02-28 06:53:59  2022-02-28 18:03:55  2022-02-28

[90 rows x 3 columns]


In [17]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 07:28:17  2022-12-01 16:41:57  2022-12-01
1   2022-12-02 07:29:21  2022-12-02 16:41:39  2022-12-02
2   2022-12-03 07:30:24  2022-12-03 16:41:24  2022-12-03
3   2022-12-04 07:31:26  2022-12-04 16:41:11  2022-12-04
4   2022-12-05 07:32:27  2022-12-05 16:41:00  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 07:00:48  2023-02-24 17:58:32  2023-02-24
86  2023-02-25 06:59:13  2023-02-25 17:59:49  2023-02-25
87  2023-02-26 06:57:37  2023-02-26 18:01:05  2023-02-26
88  2023-02-27 06:56:00  2023-02-27 18:02:21  2023-02-27
89  2023-02-28 06:54:23  2023-02-28 18:03:37  2023-02-28

[90 rows x 3 columns]


In [18]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

C:\Users\alyha\AppData\Local\Temp\ipykernel_35136\762931898.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['date'] = precip_days2['datetime'].dt.date


In [19]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,IAG,2020-12-01 03:53:00,0.2500,-SN BR,2020-12-01,2020-12-01 07:28:48,2020-12-01 16:41:48
1,IAG,2020-12-01 06:53:00,0.7600,-SN BR,2020-12-01,2020-12-01 07:28:48,2020-12-01 16:41:48
2,IAG,2020-12-01 07:53:00,0.2500,-SN BR,2020-12-01,2020-12-01 07:28:48,2020-12-01 16:41:48
3,IAG,2020-12-01 08:53:00,0.0001,-SN BR,2020-12-01,2020-12-01 07:28:48,2020-12-01 16:41:48
4,IAG,2020-12-01 09:53:00,0.2500,-SN BR,2020-12-01,2020-12-01 07:28:48,2020-12-01 16:41:48
...,...,...,...,...,...,...,...
275,IAG,2021-02-22 22:53:00,0.0001,-SN BR,2021-02-22,2021-02-22 07:03:10,2021-02-22 17:56:35
276,IAG,2021-02-22 23:53:00,0.0001,-SN BR,2021-02-22,2021-02-22 07:03:10,2021-02-22 17:56:35
277,IAG,2021-02-23 00:53:00,0.0001,-SN BR,2021-02-23,2021-02-23 07:01:37,2021-02-23 17:57:53
278,IAG,2021-02-23 11:53:00,0.0001,-SN BR,2021-02-23,2021-02-23 07:01:37,2021-02-23 17:57:53


In [20]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,IAG,2021-12-05 18:53:00,0.0001,-SN,2021-12-05,2021-12-05 07:32:42,2021-12-05 16:40:59
1,IAG,2021-12-07 04:53:00,0.0001,-SN,2021-12-07,2021-12-07 07:34:38,2021-12-07 16:40:46
2,IAG,2021-12-07 05:53:00,0.0001,-SN,2021-12-07,2021-12-07 07:34:38,2021-12-07 16:40:46
3,IAG,2021-12-07 14:53:00,0.0001,-SN,2021-12-07,2021-12-07 07:34:38,2021-12-07 16:40:46
4,IAG,2021-12-08 06:53:00,0.0001,-SN,2021-12-08,2021-12-08 07:35:34,2021-12-08 16:40:44
...,...,...,...,...,...,...,...
331,IAG,2022-02-26 18:53:00,0.0001,-SN,2022-02-26,2022-02-26 06:57:14,2022-02-26 18:01:24
332,IAG,2022-02-26 19:53:00,0.0001,-SN,2022-02-26,2022-02-26 06:57:14,2022-02-26 18:01:24
333,IAG,2022-02-26 21:53:00,0.0001,-SN,2022-02-26,2022-02-26 06:57:14,2022-02-26 18:01:24
334,IAG,2022-02-27 18:53:00,0.0001,-SN,2022-02-27,2022-02-27 06:55:37,2022-02-27 18:02:40


In [21]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,IAG,2022-12-01 07:53:00,0.0001,-SN,2022-12-01,2022-12-01 07:28:17,2022-12-01 16:41:57
1,IAG,2022-12-11 06:53:00,0.0001,-SN,2022-12-11,2022-12-11 07:38:00,2022-12-11 16:40:50
2,IAG,2022-12-11 07:53:00,0.0001,-SN,2022-12-11,2022-12-11 07:38:00,2022-12-11 16:40:50
3,IAG,2022-12-11 11:53:00,0.0001,-SN BR,2022-12-11,2022-12-11 07:38:00,2022-12-11 16:40:50
4,IAG,2022-12-11 12:53:00,0.2500,-SN BR,2022-12-11,2022-12-11 07:38:00,2022-12-11 16:40:50
...,...,...,...,...,...,...,...
167,IAG,2023-02-25 16:53:00,0.0001,-SN,2023-02-25,2023-02-25 06:59:13,2023-02-25 17:59:49
168,IAG,2023-02-25 18:53:00,0.0001,-SN,2023-02-25,2023-02-25 06:59:13,2023-02-25 17:59:49
169,IAG,2023-02-27 21:53:00,1.0200,+SN FZFG,2023-02-27,2023-02-27 06:56:00,2023-02-27 18:02:21
170,IAG,2023-02-27 22:53:00,2.2900,SN FZFG,2023-02-27,2023-02-27 06:56:00,2023-02-27 18:02:21


In [22]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
2,IAG,2020-12-01 07:53:00,0.2500,-SN BR,2020-12-01,2020-12-01 07:28:48,2020-12-01 16:41:48
3,IAG,2020-12-01 08:53:00,0.0001,-SN BR,2020-12-01,2020-12-01 07:28:48,2020-12-01 16:41:48
4,IAG,2020-12-01 09:53:00,0.2500,-SN BR,2020-12-01,2020-12-01 07:28:48,2020-12-01 16:41:48
5,IAG,2020-12-01 10:53:00,0.0001,-SN BR,2020-12-01,2020-12-01 07:28:48,2020-12-01 16:41:48
6,IAG,2020-12-01 11:53:00,0.0001,-SN BR,2020-12-01,2020-12-01 07:28:48,2020-12-01 16:41:48
...,...,...,...,...,...,...,...
272,IAG,2021-02-22 11:53:00,0.0001,-SN BR,2021-02-22,2021-02-22 07:03:10,2021-02-22 17:56:35
273,IAG,2021-02-22 12:53:00,0.0001,-SN BR,2021-02-22,2021-02-22 07:03:10,2021-02-22 17:56:35
274,IAG,2021-02-22 13:53:00,0.0001,-SN BR,2021-02-22,2021-02-22 07:03:10,2021-02-22 17:56:35
278,IAG,2021-02-23 11:53:00,0.0001,-SN BR,2021-02-23,2021-02-23 07:01:37,2021-02-23 17:57:53


In [23]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_35136\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_35136\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_35136\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [24]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
3,IAG,2020-12-01 08:53:00,0.0001,-SN BR,2020-12-01,2020-12-01 07:28:48,2020-12-01 16:41:48,2020-12-01 08:28:48,2020-12-01 15:41:48
4,IAG,2020-12-01 09:53:00,0.2500,-SN BR,2020-12-01,2020-12-01 07:28:48,2020-12-01 16:41:48,2020-12-01 08:28:48,2020-12-01 15:41:48
5,IAG,2020-12-01 10:53:00,0.0001,-SN BR,2020-12-01,2020-12-01 07:28:48,2020-12-01 16:41:48,2020-12-01 08:28:48,2020-12-01 15:41:48
6,IAG,2020-12-01 11:53:00,0.0001,-SN BR,2020-12-01,2020-12-01 07:28:48,2020-12-01 16:41:48,2020-12-01 08:28:48,2020-12-01 15:41:48
7,IAG,2020-12-01 12:53:00,0.0001,-SN BR,2020-12-01,2020-12-01 07:28:48,2020-12-01 16:41:48,2020-12-01 08:28:48,2020-12-01 15:41:48
...,...,...,...,...,...,...,...,...,...
272,IAG,2021-02-22 11:53:00,0.0001,-SN BR,2021-02-22,2021-02-22 07:03:10,2021-02-22 17:56:35,2021-02-22 08:03:10,2021-02-22 16:56:35
273,IAG,2021-02-22 12:53:00,0.0001,-SN BR,2021-02-22,2021-02-22 07:03:10,2021-02-22 17:56:35,2021-02-22 08:03:10,2021-02-22 16:56:35
274,IAG,2021-02-22 13:53:00,0.0001,-SN BR,2021-02-22,2021-02-22 07:03:10,2021-02-22 17:56:35,2021-02-22 08:03:10,2021-02-22 16:56:35
278,IAG,2021-02-23 11:53:00,0.0001,-SN BR,2021-02-23,2021-02-23 07:01:37,2021-02-23 17:57:53,2021-02-23 08:01:37,2021-02-23 16:57:53


In [25]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [26]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3


In [27]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 0.05130707070707071
2021-2022: 0.09516749999999999
2022-2023: 0.06201228070175439


In [28]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,IAG,2020-12-01 03:53:00,0.2500,-SN BR,2020-12-01,2020-12-01 07:28:48,2020-12-01 16:41:48
1,IAG,2020-12-01 06:53:00,0.7600,-SN BR,2020-12-01,2020-12-01 07:28:48,2020-12-01 16:41:48
11,IAG,2020-12-01 16:53:00,0.0001,-SN BR,2020-12-01,2020-12-01 07:28:48,2020-12-01 16:41:48
12,IAG,2020-12-01 17:53:00,0.5100,-SN BR,2020-12-01,2020-12-01 07:28:48,2020-12-01 16:41:48
13,IAG,2020-12-01 18:53:00,0.2500,-SN BR,2020-12-01,2020-12-01 07:28:48,2020-12-01 16:41:48
...,...,...,...,...,...,...,...
269,IAG,2021-02-20 23:53:00,0.0001,-SN,2021-02-20,2021-02-20 07:06:15,2021-02-20 17:54:00
270,IAG,2021-02-21 00:53:00,0.0001,-SN,2021-02-21,2021-02-21 07:04:43,2021-02-21 17:55:18
275,IAG,2021-02-22 22:53:00,0.0001,-SN BR,2021-02-22,2021-02-22 07:03:10,2021-02-22 17:56:35
276,IAG,2021-02-22 23:53:00,0.0001,-SN BR,2021-02-22,2021-02-22 07:03:10,2021-02-22 17:56:35


In [29]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_35136\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_35136\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_35136\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [30]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [31]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [32]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.01933544303797468
2021-2022: 0.017256756756756756
2022-2023: 0.19432340425531916


In [33]:
data = pd.concat([days1, nights1, days2, nights2, days3, nights3])

data['date'] = pd.to_datetime(data['date'])
data['month'] = data['date'].dt.month

data['state'] = 'NY'

data.to_csv('SO.iag.csv', index=False)